In [1]:
import os
import tensorflow as tf 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import layers, models

imgHeight, imgWidth = 256, 256
batchSize = 32

trainDir = r"Dataset/Train"
valDir = r"Dataset/Validation"
checkpointDir = r"./Checkpoints"

In [ ]:
# Data augmentation & normalization
trainDatagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

valDatagen = ImageDataGenerator(rescale=1./255)

# Load dataset
trainGenerator = trainDatagen.flow_from_directory(
    trainDir,
    target_size=(imgHeight, imgWidth),
    batch_size=batchSize,
    class_mode='binary'
)

valGenerator = valDatagen.flow_from_directory(
    valDir,
    target_size=(imgHeight, imgWidth),
    batch_size=batchSize,
    class_mode='binary'
)

checkpointCallback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpointDir, 'epoch_{epoch:02d}.keras'),
    save_weights_only=False,
    save_freq='epoch',                
    verbose=1
)


In [ ]:
# CNN Model
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', input_shape=(imgHeight, imgWidth, 3)),
    layers.MaxPooling2D((2,2)),
    
    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Flatten(),
    layers.Dropout(0.5),
    layers.Dense(512, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # Binary classification
])

# Compilation
model.compile(
    loss='binary_crossentropy',
    optimizer='adam',
    metrics=['accuracy']
)

# Training
history = model.fit(
    trainGenerator,
    epochs=10,
    validation_data=valGenerator,
    callbacks=checkpointCallback
)

model.save('deepfake_cnn_model.h5')

In [ ]:
#########################################
# RUN IF RESTARTING FROM PREVIOUS EPOCH #
#########################################

# Input path to last saved checkpoint
model = tf.keras.models.load_model(r"Checkpoints/epoch_04.keras")

trainDatagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)

valDatagen = ImageDataGenerator(rescale=1./255)

trainGenerator = trainDatagen.flow_from_directory(
    trainDir,
    target_size=(imgHeight, imgWidth),
    batch_size=batchSize,
    class_mode='binary'
)

valGenerator = valDatagen.flow_from_directory(
    valDir,
    target_size=(imgHeight, imgWidth),
    batch_size=batchSize,
    class_mode='binary'
)

checkpointCallback = tf.keras.callbacks.ModelCheckpoint(
    filepath=os.path.join(checkpointDir, 'epoch_{epoch:02d}.keras'),
    save_weights_only=False,      
    save_freq="epoch",           
    monitor="val_loss",
    verbose=1
)

history = model.fit(
    trainGenerator,
    epochs=10,          
    initial_epoch=4, # Change based on last saved checkpoint
    validation_data=valGenerator,
    callbacks=checkpointCallback
)

model.save('deepfake_cnn_model.h5')